In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
tmax_data=pd.read_excel('/content/drive/MyDrive/1985-2014_data_tmax.xlsx')

In [ ]:
tmax_data

,year,month,date,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 1159,Unnamed: 1160,Unnamed: 1161,Unnamed: 1162,Unnamed: 1163,Unnamed: 1164,Unnamed: 1165,Unnamed: 1166,Unnamed: 1167,Unnamed: 1168
0,1985,1,1,28.96,28.65,28.64,29.06,23.81,25.19,24.41,...,28.48,22.93,27.83,28.37,28.12,27.45,27.63,27.74,25.39,27.42
1,1985,1,2,29.30,28.97,28.92,29.54,24.25,25.60,24.89,...,28.60,23.11,28.07,28.69,28.46,27.81,28.03,28.22,25.91,27.95
2,1985,1,3,29.80,29.44,29.38,30.31,25.01,26.35,25.62,...,28.94,23.47,28.47,29.11,28.87,28.21,28.45,28.69,26.40,28.44
3,1985,1,4,29.35,29.06,29.05,29.82,24.61,26.02,25.32,...,27.74,22.20,27.10,27.67,27.42,26.80,27.01,27.17,24.82,26.90
4,1985,1,5,29.56,29.28,29.30,30.25,25.05,26.46,25.78,...,28.05,22.54,27.48,28.09,27.88,27.28,27.51,27.68,25.36,27.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10945,2014,12,27,30.77,30.59,30.60,31.90,26.78,28.27,27.64,...,29.22,23.62,28.51,29.07,28.71,27.95,28.13,28.30,25.95,27.95
10946,2014,12,28,30.47,30.30,30.32,31.58,26.48,27.99,27.38,...,29.12,23.53,28.41,28.97,28.62,27.87,28.04,28.20,25.85,27.85
10947,2014,12,29,30.41,30.25,30.28,31.48,26.37,27.87,27.26,...,29.08,23.48,28.37,28.92,28.57,27.82,28.00,28.17,25.83,27.84
10948,2014,12,30,30.43,30.28,30.31,31.50,26.40,27.90,27.30,...,29.16,23.57,28.43,28.97,28.61,27.86,28.01,28.17,25.81,27.81


In [ ]:
import pandas as pd
import numpy as np



# Extract the temperature columns
tmax_cols = [col for col in tmax_data.columns if col.startswith('unnamed')]

# Set the base period
base_period = [1985, 2014]

# Select the temperature data for the base period
tmax_base = tmax_data[(tmax_data['year'] >= base_period[0]) & (tmax_data['year'] <= base_period[1])]

# Select the rows where year is 2014
tmax_2014 = tmax_base[tmax_base['year'] == 2014]

# Add new column with the 10th percentile of each location
for col in tmax_cols:
    tmax_2014[col+'_tn10p'] = np.percentile(tmax_2014[col], 10)

# Convert the dataframe to dataframe again
tmax_2014 = pd.DataFrame(tmax_2014)

# Save the results to a CSV file
tmax_2014.to_csv('tx10p_of_2014.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np

# Load the temperature data into a DataFrame

# Select the temperature data for the year 2014
tmax_2014 = tmax_data[tmax_data['year'] == 2014]

# Calculate the 90th percentile for each location
tmax_90p = tmax_2014.loc[:, tmax_2014.columns[3:]].apply(lambda x: np.percentile(x, 90))

# Create a new DataFrame to store the WSDI results
wsdi_data = pd.DataFrame(columns=tmax_2014.columns[3:])

# Iterate through each location
for col in tmax_2014.columns[3:]:
    # Select the temperature data for the current location
    location_data = tmax_2014[col]
    
    # Initialize a variable to store the number of warm spell days
    warm_spell_days = 0
    
    # Initialize a variable to store the consecutive warm spell days
    consecutive_days = 0
    
    # Iterate through each day in the temperature data
    for day in location_data:
        # Check if the temperature is above the 90th percentile
        if day > tmax_90p.loc[col]:
            # Increment the number of consecutive warm spell days
            consecutive_days += 1
            
            # Check if there are at least 6 consecutive warm spell days
            if consecutive_days >= 6:
                # Increment the number of warm spell days
                warm_spell_days += 1
                
                # Reset the consecutive warm spell days
                consecutive_days = 0
        else:
            # Reset the consecutive warm spell days
            consecutive_days = 0
    
    # Add the warm spell days to the WSDI DataFrame
    wsdi_data.loc[col, 'wsdi'] = warm_spell_days

# Save the WSDI results to a CSV file
# Add a new row to the WSDI dataframe
wsdi_data.loc['WSDI'] = wsdi_data.sum()

wsdi_data.to_csv('wsdi_2014.csv', index=False)
